In [ ]:
import os

import pandas as pd

path_to_folder = "/home/jupyter/datasphere/project"

df = pd.read_csv(os.path.join(path_to_folder, "df_lin.csv"))
df

,Unnamed: 0,PA,PC,PA_DP,PC_DP,PC_pH,NaCl,phi,Vetv_block,Vetvl_liof,PA_idx,PC_idx,eps,eps1,eps2,eps3,PA_MaxAbsEStateIndex,PA_MaxEStateIndex,PA_MinAbsEStateIndex,PA_MinEStateIndex,PA_qed,PA_SPS,PA_MolWt,PA_HeavyAtomMolWt,PA_ExactMolWt,PA_NumValenceElectrons,PA_FpDensityMorgan1,PA_FpDensityMorgan2,PA_FpDensityMorgan3,PA_AvgIpc,PA_BalabanJ,PA_BertzCT,PA_Chi0,PA_Chi0n,PA_Chi0v,PA_Chi1,PA_Chi1n,PA_Chi1v,PA_Chi2n,PA_Chi2v,...,PС_EState_VSA6,PС_EState_VSA7,PС_EState_VSA8,PС_VSA_EState1,PС_VSA_EState2,PС_VSA_EState3,PС_VSA_EState4,PС_VSA_EState5,PС_VSA_EState6,PС_VSA_EState7,PС_VSA_EState8,PС_VSA_EState9,PС_FractionCSP3,PС_HeavyAtomCount,PС_NOCount,PС_NumAromaticHeterocycles,PС_NumAromaticRings,PС_NumHAcceptors,PС_NumHeteroatoms,PС_NumRotatableBonds,PС_RingCount,PС_MolLogP,PС_MolMR,PС_fr_Al_COO,PС_fr_Ar_N,PС_fr_COO,PС_fr_COO2,PС_fr_C_O,PС_fr_C_O_noCOO,PС_fr_NH0,PС_fr_Ndealkylation1,PС_fr_aryl_methyl,PС_fr_pyridine,PС_fr_quatN,PС_fr_unbrch_alkane,PA_canon,PC_canon,PA_embed,PC_embed,CONC
0,0,[*]C(C[*])C(=O)[O-],[*][N+](C)(C)CCC[N+](C)(C)CCC[*],30,25,9.0,5,63,0.0,0,1,4,0.833333,0,1,0,9.921713,9.921713,0.037037,-0.782407,0.420423,19.2,71.055,68.031,71.013853,28.0,2.6,3.2,3.2,1.645711,3.343417,73.335159,5.861807,2.600954,2.600954,3.180739,1.105172,1.105172,0.523160,0.523160,...,6.923737,0.0,28.190688,2.377749,1.353395,0.0,0.000000,0.0,0.000000,2.901697,5.783797,9.583362,1.000000,12.0,2.0,0.0,0.0,0.0,4.0,7.0,0.0,1.3513,52.8588,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,[*]CC([*])C(=O)[O-],[*]CCC[N+](C)(C)CCC[N+]([*])(C)C,"[0.55095255, 0.667746, 0.3779173, -1.1744181, ...","[0.015644865, 0.14808306, -0.0503309, -0.07997...",[ 0.55095255 0.667746 0.3779173 ... -0.22...
1,1,[*]C(C[*])C(=O)[O-],[*][N+](C)(C)CCC[N+](C)(C)CCC[*],30,30,9.0,5,63,0.0,0,1,4,1.000000,0,0,1,9.921713,9.921713,0.037037,-0.782407,0.420423,19.2,71.055,68.031,71.013853,28.0,2.6,3.2,3.2,1.645711,3.343417,73.335159,5.861807,2.600954,2.600954,3.180739,1.105172,1.105172,0.523160,0.523160,...,6.923737,0.0,28.190688,2.377749,1.353395,0.0,0.000000,0.0,0.000000,2.901697,5.783797,9.583362,1.000000,12.0,2.0,0.0,0.0,0.0,4.0,7.0,0.0,1.3513,52.8588,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,[*]CC([*])C(=O)[O-],[*]CCC[N+](C)(C)CCC[N+]([*])(C)C,"[0.55095255, 0.667746, 0.3779173, -1.1744181, ...","[0.015644865, 0.14808306, -0.0503309, -0.07997...",[ 0.55095255 0.667746 0.3779173 ... -0.22...
2,2,[*]C(C[*])C(=O)[O-],[*][N+](C)(C)CCC[N+](C)(C)CCC[*],30,60,9.0,5,63,0.0,0,1,4,2.000000,1,0,0,9.921713,9.921713,0.037037,-0.782407,0.420423,19.2,71.055,68.031,71.013853,28.0,2.6,3.2,3.2,1.645711,3.343417,73.335159,5.861807,2.600954,2.600954,3.180739,1.105172,1.105172,0.523160,0.523160,...,6.923737,0.0,28.190688,2.377749,1.353395,0.0,0.000000,0.0,0.000000,2.901697,5.783797,9.583362,1.000000,12.0,2.0,0.0,0.0,0.0,4.0,7.0,0.0,1.3513,52.8588,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,[*]CC([*])C(=O)[O-],[*]CCC[N+](C)(C)CCC[N+]([*])(C)C,"[0.55095255, 0.667746, 0.3779173, -1.1744181, ...","[0.015644865, 0.14808306, -0.0503309, -0.07997...",[ 0.55095255 0.667746 0.3779173 ... -0.22...
3,3,[*]C(C[*])C(=O)[O-],[*][N+](C)(C)CCC[N+](C)(C)CCC[*],30,65,9.0,5,63,0.0,0,1,4,2.166667,1,0,0,9.921713,9.921713,0.037037,-0.782407,0.420423,19.2,71.055,68.031,71.013853,28.0,2.6,3.2,3.2,1.645711,3.343417,73.335159,5.861807,2.600954,2.600954,3.180739,1.105172,1.105172,0.523160,0.523160,...,6.923737,0.0,28.190688,2.377749,1.353395,0.0,0.000000,0.0,0.000000,2.901697,5.783797,9.583362,1.000000,12.0,2.0,0.0,0.0,0.0,4.0,7.0,0.0,1.3513,52.8588,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,[*]CC([*])C(=O)[O-],[*]CCC[N+](C)(C)CCC[N+]([*])(C)C,"[0.55095255, 0.667746, 0.3779173, -1.1744181, ...","[0.015644865, 0.14808306, -0.0503309, -0.07997...",[ 0.55095255 0.667746 0.3779173 ... -0.22...
4,4,[*]C(C[*])C(=O)[O-],[*][N+](C)(C)CCC[N+](C)(C)CCC[*],30,80,9.0,5,63,0.0,0,1,4,2.666667,1,0,0,9.921713,9.921713,0.037037,-0.782407,0.420423,19.2,71.055,68.031,71.013853,28.0,2.6,3.2,3.2,1.645711,3.343417,73.335159,5.861807,2.6009

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
list_features = df.select_dtypes(exclude=['object']).columns.tolist()


df_test = pd.concat([df.loc[(df["PA_idx"] == 1) & (df["PC_idx"] == 3)].copy(), df.loc[(df["PA_idx"] == 4) & (df["PC_idx"] == 2)].copy()])# index_to_drop = df_test.index
df_train_all = df.drop(index=df_test.index)  # Sample 20% for the validation set
df_train = df_train_all.sample(frac=0.8, random_state = 42)  # Sample 20% for the test set
df_val = df_train_all.drop(index=df_train.index)  # Sample

scaler = StandardScaler()

df_train_scaled = pd.DataFrame(
    scaler.fit_transform(df_train[list_features]), columns=list_features
)
df_val_scaled = pd.DataFrame(
    scaler.transform(df_val[list_features]), columns=list_features
)
df_test_scaled = pd.DataFrame(
    scaler.transform(df_test[list_features]), columns=list_features
)


# Scale the target variable "phi" using StandardScaler
scaler_phi = StandardScaler()
df_train_scaled["phi"] = scaler_phi.fit_transform(df_train[["phi"]]).flatten()
df_val_scaled["phi"] = scaler_phi.transform(df_val[["phi"]]).flatten()
df_test_scaled["phi"] = scaler_phi.transform(df_test[["phi"]]).flatten()


# Reset the indices of the DataFrames before adding the columns
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

# Reset the indices of the scaled DataFrames
df_train_scaled.reset_index(drop=True, inplace=True)
df_val_scaled.reset_index(drop=True, inplace=True)
df_test_scaled.reset_index(drop=True, inplace=True)

# Add ["PA_canon", "PC_canon"] columns
# df_train_scaled = pd.concat([df_train_scaled, df_train[["PA_canon", "PC_canon"]]], axis=1)
df_train_scaled["PA_canon"] = df_train["PA_canon"]
df_val_scaled["PA_canon"] = df_val["PA_canon"]
df_test_scaled["PA_canon"] = df_test["PA_canon"]

df_train_scaled["PC_canon"] = df_train["PC_canon"]
df_val_scaled["PC_canon"] = df_val["PC_canon"]
df_test_scaled["PC_canon"] = df_test["PC_canon"]

In [ ]:
print("Train set shape:", df_train_scaled.shape)
print("Test set shape:", df_test_scaled.shape)
print("Validation set shape:", df_val_scaled.shape)

157
73
39
269


In [ ]:
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset


class CustomDatasetBERT_upd(Dataset):
    def __init__(self, df, list_features):
        self.df = df

        self.list_features = list_features

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        inp_PA = row["PA_canon"]
        inp_PC = row["PC_canon"]

        features = np.array(row[self.list_features], dtype=np.float32)

        concatenated_features = np.concatenate(features, axis=None)

        target = torch.tensor(row["phi"], dtype=torch.float32)

        return inp_PA, inp_PC, concatenated_features, target

In [ ]:
import torch

train_set = CustomDatasetBERT_upd(df_train_scaled, list_features)
val_set = CustomDatasetBERT_upd(df_val_scaled, list_features)
test_set = CustomDatasetBERT_upd(df_test_scaled, list_features)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=8, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False)

In [ ]:
%pip install -q lion-pytorch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import lightning as L
import torch.nn.functional
import torchmetrics
from torch import optim

from lion_pytorch import Lion

class LitBasic_upd(L.LightningModule):
    def __init__(self, model, only_head=False):
        super().__init__()
        self.model = model
        self.only_head = only_head

        self.save_hyperparameters(logger=False)

        self.train_acc = torchmetrics.R2Score()
        self.test_acc = torchmetrics.R2Score()
        self.val_acc = torchmetrics.R2Score()

        self.crit = nn.MSELoss()

        self.training_step_outputs = []
        self.test_step_outputs = []
        self.validation_step_outputs = []

    def configure_optimizers(self):
        if self.only_head:
            for param in self.model.parameters():
                param.requires_grad = False

            for param in self.model.feature_net.parameters():
                param.requires_grad = True
            for param in self.model.layers_stack_chem.parameters():
                param.requires_grad = True
            for param in self.model.layers_stack_all.parameters():
                param.requires_grad = True

            optimizer = Lion(filter(lambda p: p.requires_grad, self.model.parameters()), lr=1e-4 , weight_decay=1e-3)

            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, factor=0.5
            )

        else:
            for param in self.model.parameters():
                param.requires_grad = True

            optimizer = Lion(self.model.parameters(), lr=1e-8, weight_decay=1e-3)


            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, factor=0.5
            )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
        }

    def training_step(self, batch, batch_idx):
        x, y, concatenated_features, target = batch

        pred = self.model(x, y, concatenated_features)
        target = target.view(-1)
        pred = pred.view(-1)
        train_loss = self.crit(pred, target)
        self.log("train_loss", train_loss)
        self.train_acc.update(pred, target)

        return train_loss

    def validation_step(self, batch, batch_idx):

        x, y, concatenated_features, target = batch
        pred = self.model(x, y, concatenated_features)

        target = target.view(-1)
        pred = pred.view(-1)
        val_loss = self.crit(pred, target)
        self.log("val_loss", val_loss)

        self.val_acc.update(pred, target)

    def test_step(self, batch, batch_idx):

        x, y, concatenated_features, target = batch

        pred = self.model(x, y, concatenated_features)
        target = target.view(-1)
        pred = pred.view(-1)
        test_loss = self.crit(pred, target)
        self.test_acc.update(pred, target)

    def on_train_epoch_end(self):

        acc_value_train = self.train_acc.compute()
        self.log("r2/train", acc_value_train)
        self.train_acc.reset()

    def on_validation_epoch_end(self):

        acc_value_valid = self.val_acc.compute()
        self.log("r2/val", acc_value_valid)
        self.val_acc.reset()

    def on_test_epoch_end(self):

        acc_value_test = self.test_acc.compute()
        self.log("r2/test: ", acc_value_test)
        self.test_acc.reset()

In [ ]:
# from L.pytorch.callbacks import ModelCheckpoint  # - doesn't work
from lightning.pytorch.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    # dirpath="https://disk.yandex.ru/d/Q40jIIToIod6HA",
    dirpath="/home/jupyter/datasphere/project/Data/Lion_vse_struct_42_500",
    save_last=True,
    every_n_epochs=1,
    save_top_k=1,
    save_weights_only=False,
    monitor="r2/val",
    mode="max",
)

In [ ]:
import lightning as L
import numpy as np
import torch
from pytorch_lightning.loggers import TensorBoardLogger

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


np.random.seed(42)

L.seed_everything(42)


lit_model = LitBasic_upd(model_upd, only_head=False)

trainer = L.Trainer(
    max_epochs=300,
    callbacks=[checkpoint_callback],
    log_every_n_steps=2,
    logger=TensorBoardLogger(save_dir="/home/jupyter/datasphere/project/Data/Lion_vse_struct_42_500"),
)

trainer.fit(model=lit_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


Seed set to 42
/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /home/jupyter/work/resources/Data/Lion_vse_struct_42_500 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params
-------------------------------------------
0 | model     | IPEC_net_upd | 25.4 M
1 | train_acc | R2Score      | 0     
2 | test_acc  | R2Score      | 0     
3 | val_acc   | R2Score      | 0     
4 | crit      | MSELoss      | 0     
-----------------------------------------

Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 39.07it/s]

/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 20/20 [00:01<00:00, 12.73it/s, v_num=1]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 20/20 [00:01<00:00, 12.76it/s, v_num=1]     
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 20/20 [00:01<00:00, 12.60it/s, v_num=1]     
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 20/20 [00:01<00:00, 12.76it/s, v_num=1]     
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 20/20 [00:01<00:00, 12.61it/s, v_num=1]     
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 20/20 [00:01<00:00, 12.92it/s, v_num=1]     
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|██████████| 20/20 [00:01<00:00, 12.70it/s, v_num=1]     
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|██████████| 20/20 [00:01<00:00, 12.79it/s, v_num=1]     
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 8: 100%|██████████| 20/20 [00:01<00:00, 12.75it/s, v_num=1]    

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 20/20 [00:03<00:00,  5.89it/s, v_num=1]


In [ ]:
# mc_to_tb - проблема, Needs at least two samples to calculate r2 score.

# Валидация

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir /home/jupyter/datasphere/project/Data/Lion_vse_struct_42_500 --load_fast=false --port=7007
# port надо другой брать и тогда всё будет хорошо

In [ ]:
trainer.test(lit_model, dataloaders=test_loader, ckpt_path="best")
# test_strange_loader

Restoring states from the checkpoint path at /home/jupyter/work/resources/Data/Lion_vse_struct_42_500/epoch=84-step=1700.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/jupyter/work/resources/Data/Lion_vse_struct_42_500/epoch=84-step=1700.ckpt
/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 34.17it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         r2/test:          │    0.7818993330001831     │
└───────────────────────────┴───────────────────────────┘

[{'r2/test: ': 0.7818993330001831}]